In [1]:
import numpy as np
import math

import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cfeature
import matplotlib.colors as colors
from matplotlib.cm import get_cmap
from matplotlib import ticker
import matplotlib.gridspec as gridspec

from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point

from IPython.display import Image

from tqdm import tqdm
import os 
from datetime import date

g = 9.8

In [2]:
Year = "2007"

Months = []
for imon in range(1,13): ### shw
    Months.append(str(imon).zfill(2))
    
Days = []
for iday in range(1,30,3):
    Days.append(str(iday).zfill(2))
    
print(Year)
print(Months)
print(Days)

2007
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
['01', '04', '07', '10', '13', '16', '19', '22', '25', '28']


In [3]:
# directory = '/n/home12/hongwei/HONGWEI/lagranto_era5_0.2um/Simulation_0.2um/'+Year+'/'
directory = '/n/home12/hongwei/HONGWEI/lagranto_era5_0.2um/Simulation_0.2um_Publication/6_altitudes/'+Year+'/'


N_head = 5 # first 5 lines are head lines, not include any useful data
N_column = 4

# Nx = 36
# Ny = 21
# Nz = 5
# N_parcel = 3780

Nx = 24
Ny = 21
Nz = 6
N_parcel = Nx*Ny*Nz

if Nx*Ny*Nz!=N_parcel: print('ERROR: parcel number is wrong!')

In [4]:
# loop for all traj files

for month in Months:
    for day in tqdm(Days):
    
        # (1) read original data from traj files
        filename = "traj_"+Year+month+day+".1"
        file1 = open(directory+filename, 'r')
        Lines = file1.readlines()

        Nt = int( (len(Lines)-4)/N_parcel - 1 )
                
        data = np.zeros((N_parcel, Nt, N_column))

        count = 0
        # Strips the newline character
        for line in Lines:
            count += 1
    
            if count>=5:
                i = count-5
                i_parcel = math.floor( i / (Nt+1) ) # Nt time lines plue 1 empty line
                i_t = i%(Nt+1)
        
                if i_t!=0:
                    a = line.split()

                    if i_t==1:
                        if float(a[0])!=0.0: print('ERROR: first time is not 0 !!!')
                
                    data[i_parcel,i_t-1,0] = float(a[0]) # t [h]
                    data[i_parcel,i_t-1,1] = float(a[1]) # lon
                    data[i_parcel,i_t-1,2] = float(a[2]) # lat
                    data[i_parcel,i_t-1,3] = float(a[3]) # lev

                                    
        # (2) re-shape data(N_parcel, Nt, N_column) to data2(Nz,Ny,Nx,Nt,N_column)
        data2 = np.zeros((Nz,Ny,Nx,Nt,N_column))

        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    i_parcel = ix*(Ny*Nz) + iy*Nz + iz
                    data2[iz,iy,ix,:,:] = data[i_parcel,:,:]
                
        
        file1.close()
        
        
        
        # save 6-monly location
        # lat
        with open('./6_monthly_location/xyz_'+Year+month+day+'.txt', 'w') as f1:
            for ix in range(Nx):
                for iy in range(Ny):
                    for iz in range(Nz):
                                                
                        f1.write( str(data2[iz,iy,ix,0  ,1]) + ',' \
                                + str(data2[iz,iy,ix,0  ,2]) + ',' \
                                + str(data2[iz,iy,ix,0  ,3]) + ',' \
                                 
                                + str(data2[iz,iy,ix,30 ,1]) + ',' \
                                + str(data2[iz,iy,ix,30 ,2]) + ',' \
                                + str(data2[iz,iy,ix,30 ,3]) + ',' \
                                 
                                + str(data2[iz,iy,ix,60 ,1]) + ',' \
                                + str(data2[iz,iy,ix,60 ,2]) + ',' \
                                + str(data2[iz,iy,ix,60 ,3]) + ',' \

                                + str(data2[iz,iy,ix,90 ,1]) + ',' \
                                + str(data2[iz,iy,ix,90 ,2]) + ',' \
                                + str(data2[iz,iy,ix,90 ,3]) + ',' \
                                 
                                + str(data2[iz,iy,ix,120,1]) + ',' \
                                + str(data2[iz,iy,ix,120,2]) + ',' \
                                + str(data2[iz,iy,ix,120,3]) + ',' \
                                 
                                + str(data2[iz,iy,ix,150,1]) + ',' \
                                + str(data2[iz,iy,ix,150,2]) + ',' \
                                + str(data2[iz,iy,ix,150,3]) + ',' \
                                 
                                + str(data2[iz,iy,ix,180,1]) + ',' \
                                + str(data2[iz,iy,ix,180,2]) + ',' \
                                + str(data2[iz,iy,ix,180,3])  )
                        
                        f1.write('\n')
        f1.close()
        


100%|██████████| 10/10 [04:04<00:00, 24.44s/it]
